In [11]:
!pip install hyperopt

     ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.6 MB ? eta -:--:--
     --- ------------------------------------ 0.1/1.6 MB 1.2 MB/s eta 0:00:02
     --------------- ------------------------ 0.6/1.6 MB 4.4 MB/s eta 0:00:01
     ------------------------------- -------- 1.3/1.6 MB 6.6 MB/s eta 0:00:01
     ---------------------------------------- 1.6/1.6 MB 6.7 MB/s eta 0:00:00
     ---------------------------------------- 0.0/200.5 kB ? eta -:--:--
     ------------------------------------- 200.5/200.5 kB 11.9 MB/s eta 0:00:00


DEPRECATION: fairness 0.1.8 has a non-standard dependency specifier BlackBoxAuditing>=0.1.26ggplot. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of fairness or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [24]:
!pip install tpot

     ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
     ---------------------------------------- 0.0/1.1 MB ? eta -:--:--
     ------ --------------------------------- 0.2/1.1 MB 2.6 MB/s eta 0:00:01
     ------------------- -------------------- 0.5/1.1 MB 4.6 MB/s eta 0:00:01
     ------------------------------- -------- 0.8/1.1 MB 5.4 MB/s eta 0:00:01
     ---------------------------------------- 1.1/1.1 MB 5.6 MB/s eta 0:00:00
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
  Preparing metadata (setup.py): started
  Preparing metadata (setup.py): finished with status 'done'
   ---------------------------------------- 0.0/87.4 kB ? eta -:--:--
   ---------------------------------------- 87.4/87.4 kB 4.8 MB/s eta 0:00:00
   ---------------------------------------- 0.0/99.8 MB ? eta -:--:--
   ---------------------------------------- 0.5/99.8 MB 14.5 MB/s eta 0:00:07
   ---------------------------------------- 1.

DEPRECATION: fairness 0.1.8 has a non-standard dependency specifier BlackBoxAuditing>=0.1.26ggplot. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of fairness or contact the author to suggest that they release a version with a conforming dependency specifiers. Discussion can be found at https://github.com/pypa/pip/issues/12063


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

creadit_data = pd.read_csv('../Hyperparameter Tuning/Dataset/credit-card-full.csv')

In [2]:
creadit_data.head()

,ID,LIMIT_BAL,SEX,EDUCATION,MARRIAGE,AGE,PAY_0,PAY_2,PAY_3,PAY_4,...,BILL_AMT4,BILL_AMT5,BILL_AMT6,PAY_AMT1,PAY_AMT2,PAY_AMT3,PAY_AMT4,PAY_AMT5,PAY_AMT6,default payment next month
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


In [4]:
X = creadit_data.drop('default payment next month', axis=1)
y = creadit_data['default payment next month']

In [5]:
from sklearn.model_selection import train_test_split

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y, stratify=y, random_state=42)

# Coarse to Fine Tuning

This is a markdown cell used to describe and document the workflow.

## A basic informed search methodology:

Start out with a rough, random approach and iteratively refine your search.

The process is:

1. Random search
2. Find promising areas
3. Grid search in the smaller area
4. Continue until optimal score obtained

You could substitute (3) with further random searches before the grid search
You could also substitute step 3 with further random searches before the grid search.


In [8]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV

## Bayes in Hyperparameter Tuning

We can apply this logic to hyperparameter tuning:

1. Pick a hyperparameter combination
2. Build a model
3. Get new evidence (the score of the model)
4. Update our beliefs and choose better hyperparameters next round

Bayesian hyperparameter tuning is very new but quite popular for larger and more complex hyperparameter tuning tasks as they work well to find optimal hyperparameter combinations in these situations.
We can apply this logic to hyperparameter tuning:

. Pick a hyperparameter combination

. Build a model

. Get new evidence (the score of the model)
. Update our beliefs and chose better hyperparameters next round
Bayesian hyperparameter tuning is very new but quite popular for larger and more complex
hyperparameter tuning tasks as they work well to find optimal hyperparameter combinations
in these situations

Bayesian hyperparameter tuning is very new but quite popular for larger and more
complex hyperparameter tuning tasks.

In [12]:
from hyperopt import hp

In [13]:

space = {
    'max_depth': hp.quniform('max_depth', 2, 10, 2),
    'min_samples_leaf': hp.uniform('min_samples_leaf', 2, 8, 2),
    'learning_rate': hp.loguniform('learning_rate', 0.01, 1, 55)
}

In [15]:
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import cross_val_score

In [21]:
def objective(params):
    params = {
        'max_depth': int(params['max_depth']),
        'min_samples_leaf': int(params['min_samples_leaf']),
        'learning_rate': params['learning_rate']
    }
    gbm_clf = GradientBoostingClassifier(n_estimators=100, **params, random_state=42)
    best_score = cross_val_score(gbm_clf, X_train, y_train, scoring='accuracy', cv=5, n_jobs=-1).mean()
    loss = 1 - best_score
    write_results(best_score, params, iteration)
    return loss

In [22]:
from hyperopt import fmin, tpe

In [23]:
best_result = fmin(
    fn=objective,
    space=space,
    max_evals=500,
    rstate=np.random.RandomState(42),
    algo=tpe.suggest
)

  0%|          | 0/500 [00:00<?, ?trial/s, best loss=?]


AttributeError: 'numpy.random.mtrand.RandomState' object has no attribute 'integers'

In genetic evolution in the real world, we have the following process:

1. There are many creatures existing ('offspring')
2. The strongest creatures survive and pair off
3. There is some 'crossover' as they form offspring
4. There are random mutations to some of the offspring
    - These mutations sometimes help give some offspring an advantage
    
5. Go back to (1)!

We can apply the same idea to hyperparameter tuning:

1. We can create some models (that have hyperparameter settings)

2. We can pick the best (by our scoring function)
. These are the ones that 'survive'

3. We can create new models that are similar to the best ones

4. We add in some randomness so we don't reach a local optimum

5. Repeat until we are happy!

The key arguments to a TPOT classifier are:

- `generations`: Iterations to run training for.
- `population_size`: The number of models to keep after each iteration.
- `offspring_size`: Number of models to produce in each iteration.
- `mutation_rate`: The proportion of pipelines to apply randomness to.
- `crossover_rate`: The proportion of pipelines to breed each iteration.
- `scoring`: The function to determine the best models.
- `cv`: Cross-validation strategy to use.


In [25]:
from tpot import TPOTClassifier


c:\Users\fahad\anaconda3\Lib\site-packages\tpot\builtins\__init__.py:36: UserWarning: Warning: optional dependency `torch` is not available. - skipping import of NN models.
  warnings.warn("Warning: optional dependency `torch` is not available. - skipping import of NN models.")


In [26]:
tpot = TPOTClassifier(
    generations=3,
    population_size=5,
    verbosity=2,
    offspring_size=10,
    scoring='accuracy',
    cv=5,
    random_state=42)

In [28]:
tpot.fit(X_train, y_train)

Optimization Progress:   0%|          | 0/35 [00:00<?, ?pipeline/s]


Generation 1 - Current best internal CV score: 0.8216888888888889

Generation 2 - Current best internal CV score: 0.8220444444444445

Generation 3 - Current best internal CV score: 0.8220444444444445

Best pipeline: ExtraTreesClassifier(input_matrix, bootstrap=True, criterion=entropy, max_features=0.6000000000000001, min_samples_leaf=7, min_samples_split=3, n_estimators=100)


TPOTClassifier(generations=3, offspring_size=10, population_size=5,
               random_state=42, scoring='accuracy', verbosity=2)

In [30]:
tpot.score(X_test, y_test)

0.8172